# ProxyWhirl End-to-End Examples

Welcome! This notebook walks through the major interfaces exposed by the [ProxyWhirl](https://github.com/wyattowalsh/proxywhirl) project, including the Python library, CLI tooling, and REST API server. Each section is designed to be copy/paste friendly and provides inline commentary so you can adapt the snippets to your own environment.

> Last updated: October 30, 2025


## Table of Contents
- [0. Setup](#0-setup)
- [1. Core Library Quickstart](#1-core-library-quickstart)
- [2. Strategy Deep Dive](#2-strategy-deep-dive)
- [3. Fetching, Validation, and Metadata](#3-fetching-validation-and-metadata)
- [4. Persistence Backends](#4-persistence-backends)
- [5. CLI Workbench](#5-cli-workbench)
- [6. REST API Walkthrough](#6-rest-api-walkthrough)
- [7. Advanced Patterns & Utilities](#7-advanced-patterns--utilities)
- [8. Where to Go Next](#8-where-to-go-next)


## 0. Setup
This notebook assumes you are working inside the `proxywhirl` repository with [uv](https://github.com/astral-sh/uv) managing the environment declared in `pyproject.toml`. Sync the dependencies (including optional extras) before running the examples:

```bash
uv sync --extra storage --extra js
# or, to install just the base package from PyPI
uv pip install proxywhirl
```

The extras pull in FastAPI, Playwright support, SQLModel, and encryption helpers so every section below can execute without additional work.

In [ ]:
# Verify that ProxyWhirl is importable and report the active versions.
import platform

try:
    import proxywhirl
except ImportError as exc:  # pragma: no cover - guidance for first-time setup
    raise ImportError(
        "ProxyWhirl is not installed. Install it with `pip install -e .[storage,js]` "
        "from the repository root (or `pip install "proxywhirl[storage,js]"`) and "
        "restart the kernel."
    ) from exc

print(f"Python {platform.python_version()} on {platform.system()}")
print(f"ProxyWhirl {proxywhirl.__version__}")


## 1. Core Library Quickstart
This section introduces the core building blocks: `Proxy`, `ProxyPool`, and `ProxyWhirl`. We will keep the examples deterministic by mocking outbound HTTP traffic with `httpx.MockTransport`, so you can step through the rotation logic without depending on live proxy servers.


In [ ]:
from pydantic import SecretStr
from proxywhirl import Proxy, ProxyPool, ProxySource

# Define a small pool with geography metadata and optional credentials (kept secret via SecretStr)
demo_pool = ProxyPool(
    name="demo-pool",
    proxies=[
        Proxy(
            url="http://proxy-us.local:8000",
            source=ProxySource.USER,
            country_code="US",
            metadata={"label": "Primary US"},
        ),
        Proxy(
            url="http://proxy-de.local:8000",
            source=ProxySource.USER,
            country_code="DE",
            metadata={"label": "European node"},
        ),
        Proxy(
            url="http://proxy-sg.local:8000",
            source=ProxySource.USER,
            country_code="SG",
            metadata={"label": "APAC node"},
        ),
    ],
)

secured_proxy = Proxy(
    url="http://secure-proxy.local:9000",
    username=SecretStr("demo-user"),
    password=SecretStr("demo-pass"),
    tags={"premium", "auth"},
)
demo_pool.add_proxy(secured_proxy)

for proxy in demo_pool.proxies:
    print(f"{proxy.url:32s} source={proxy.source.value:6s} country={proxy.country_code}")
print(f"Total proxies in pool: {demo_pool.size}")


In [ ]:
import httpx
from unittest.mock import patch

from proxywhirl import ProxyWhirl
from proxywhirl.strategies import RoundRobinStrategy

# Instantiate a rotator with round-robin behaviour
rotator = ProxyWhirl(
    proxies=[proxy.model_copy(deep=True) for proxy in demo_pool.get_all_proxies()],
    strategy=RoundRobinStrategy(),
)

selection_order: list[str] = []
original_select = rotator.strategy.select

def select_with_tracking(pool, context=None):
    proxy = original_select(pool, context=context)
    selection_order.append(proxy.url)
    return proxy

rotator.strategy.select = select_with_tracking  # type: ignore[assignment]


def mock_handler(request: httpx.Request) -> httpx.Response:
    current_proxy = selection_order[-1] if selection_order else "unknown"
    payload = {
        "requested_url": str(request.url),
        "method": request.method,
        "proxy_used": current_proxy,
    }
    return httpx.Response(200, json=payload)

_original_client = httpx.Client

def build_mock_client(*args, **kwargs):
    kwargs["transport"] = httpx.MockTransport(mock_handler)
    return _original_client(*args, **kwargs)

with patch("httpx.Client", build_mock_client):
    responses = [rotator.get("https://service.example/api/status").json() for _ in range(3)]

rotator.strategy.select = original_select  # Restore original method

print("Rotation order:", " → ".join(selection_order))
print("Sample responses:")
for payload in responses:
    print(payload)

print("
Pool statistics:", rotator.get_pool_stats())


In [ ]:
from proxywhirl import HealthStatus
from proxywhirl.models import ProxyPool

# Operate on a copy so we do not disturb the earlier pool
stats_pool = ProxyPool(
    name="stats-demo",
    proxies=[proxy.model_copy(deep=True) for proxy in demo_pool.get_all_proxies()],
)

# Simulate a proxy falling behind to demonstrate pool maintenance
stats_pool.proxies[1].health_status = HealthStatus.DEAD
removed = stats_pool.clear_unhealthy()

print(f"Removed {removed} unhealthy proxies → pool size now {stats_pool.size}")
print("Source breakdown:", stats_pool.get_source_breakdown())


## 2. Strategy Deep Dive
ProxyWhirl ships with several rotation strategies that solve different production problems (fair distribution, weighting by performance, sticky sessions, and geo fencing). The snippet below evaluates each strategy against the same healthy pool.


In [ ]:
from proxywhirl import (
    HealthStatus,
    Proxy,
    ProxyPool,
    ProxySource,
    SelectionContext,
    StrategyConfig,
)
from proxywhirl.strategies import (
    GeoTargetedStrategy,
    RandomStrategy,
    RoundRobinStrategy,
    SessionPersistenceStrategy,
    WeightedStrategy,
)

strategy_pool = ProxyPool(
    name="strategy-demo",
    proxies=[
        Proxy(url="http://proxy-a.local:8000", source=ProxySource.USER, country_code="US"),
        Proxy(url="http://proxy-b.local:8000", source=ProxySource.USER, country_code="DE"),
        Proxy(url="http://proxy-c.local:8000", source=ProxySource.USER, country_code="SG"),
    ],
)

for proxy in strategy_pool.proxies:
    proxy.health_status = HealthStatus.HEALTHY
    proxy.total_requests = 10
    proxy.total_successes = 8

# Round-robin: sequential ordering
rr = RoundRobinStrategy()
rr_order = [rr.select(strategy_pool).url for _ in range(3)]

# Random: unpredictable selection
rand = RandomStrategy()
random_picks = [rand.select(strategy_pool).url for _ in range(3)]

# Weighted: bias the EU node more heavily
weighted = WeightedStrategy()
weighted.configure(StrategyConfig(weights={
    "http://proxy-b.local:8000": 5.0,
    "http://proxy-a.local:8000": 1.0,
}))
weighted_picks = [weighted.select(strategy_pool).url for _ in range(5)]

# Session persistence: same session ID keeps the same proxy
session_strategy = SessionPersistenceStrategy()
sticky_results = [
    session_strategy.select(strategy_pool, SelectionContext(session_id=session_id)).url
    for session_id in ("session-1", "session-1", "session-2", "session-1")
]

# Geo targeting: prefer proxies that match the desired country
geo_strategy = GeoTargetedStrategy()
geo_choice = geo_strategy.select(strategy_pool, SelectionContext(target_country="DE")).url

print("Round-robin order:", rr_order)
print("Random picks:", random_picks)
print("Weighted picks:", weighted_picks)
print("Session stickiness:", sticky_results)
print("Geo-targeted (DE):", geo_choice)


## 3. Fetching, Validation, and Metadata
`ProxyFetcher` can download proxy lists from many formats and optionally validate each entry. To keep things offline, we patch `httpx.AsyncClient` so every download and validation call returns deterministic data.


In [ ]:
import asyncio
import httpx
from unittest.mock import patch

from proxywhirl import Proxy, ProxySource
from proxywhirl.fetchers import ProxyFetcher, ProxySourceConfig, ProxyValidator
from proxywhirl.models import ValidationLevel

sample_proxy_list = """
# Example static list (duplicates are intentional)
http://198.51.100.1:8080
http://198.51.100.2:8080
http://198.51.100.1:8080
""".strip()

async def fetch_and_validate() -> None:
    fetcher = ProxyFetcher(
        sources=[ProxySourceConfig(url="https://example.com/mock-proxies.txt", format="text")],
        validator=ProxyValidator(level=ValidationLevel.BASIC, concurrency=4),
    )

    async def fake_get(self, url, *args, **kwargs):
        # Return the static list for both fetching and validation calls
        return httpx.Response(200, text=sample_proxy_list)

    with patch.object(httpx.AsyncClient, "get", new=fake_get):
        raw_entries = await fetcher.fetch_all(validate=True)

    proxies = [Proxy(url=item["url"], source=ProxySource.FETCHED) for item in raw_entries]

    print(f"Fetched {len(raw_entries)} unique proxies")
    for proxy in proxies:
        print(f"• {proxy.url} (source={proxy.source.value})")

asyncio.run(fetch_and_validate())


## 4. Persistence Backends
ProxyWhirl supports encrypted JSON files and a fully async SQLite backend. The examples below use temporary directories so you can experiment safely.


In [ ]:
import asyncio
import tempfile
from pathlib import Path

from proxywhirl import Proxy, ProxySource
from proxywhirl.storage import FileStorage

async def demo_file_storage() -> None:
    proxies = [
        Proxy(url="http://proxy-store-1.local:9000", source=ProxySource.USER),
        Proxy(url="http://proxy-store-2.local:9000", source=ProxySource.FETCHED),
    ]

    with tempfile.TemporaryDirectory() as tmpdir:
        storage_path = Path(tmpdir) / "proxies.json"
        storage = FileStorage(storage_path)
        await storage.save(proxies)
        loaded = await storage.load()
        print(f"Stored {len(loaded)} proxies at {storage_path}")
        for proxy in loaded:
            print(f"• {proxy.url} (source={proxy.source.value})")

asyncio.run(demo_file_storage())


In [ ]:
import asyncio
import tempfile
from pathlib import Path

from proxywhirl import HealthStatus, Proxy, ProxySource
from proxywhirl.storage import SQLiteStorage

async def demo_sqlite_storage() -> None:
    with tempfile.TemporaryDirectory() as tmpdir:
        db_path = Path(tmpdir) / "proxies.db"
        storage = SQLiteStorage(db_path)
        await storage.initialize()

        proxies = [
            Proxy(url="http://proxy-db-1.local:9000", source=ProxySource.USER, health_status=HealthStatus.HEALTHY),
            Proxy(url="http://proxy-db-2.local:9000", source=ProxySource.FETCHED, health_status=HealthStatus.DEGRADED),
        ]

        await storage.save(proxies)
        all_proxies = await storage.load()
        healthy_only = await storage.query(health_status=HealthStatus.HEALTHY.value)

        print(f"Database '{db_path.name}' contains {len(all_proxies)} proxies")
        print("Healthy query result:", [p.url for p in healthy_only])

        await storage.close()

asyncio.run(demo_sqlite_storage())


## 5. CLI Workbench
The Typer-powered CLI mirrors many library features. Use `python -m proxywhirl.cli` when developing from source so the module resolves relative imports correctly. The helper below captures command output and prints any errors so you can iterate quickly.


In [ ]:
import subprocess
import sys
from textwrap import shorten


def run_cli_command(args, output_lines=15):
    cmd = [sys.executable, "-m", "proxywhirl.cli"] + args
    print(f"$ proxywhirl {' '.join(args)}")
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.stdout:
        lines = result.stdout.strip().splitlines()
        preview = lines[:output_lines]
        print("
".join(preview))
        if len(lines) > output_lines:
            print("…")
    if result.stderr:
        print("[stderr]")
        print(result.stderr.strip())
    print(f"(exit code: {result.returncode})
")

# Feel free to adjust or add more commands once dependencies are installed.
run_cli_command(["--help"])
run_cli_command(["--no-lock", "config", "show"], output_lines=20)
run_cli_command(["--no-lock", "pool", "list"])


## 6. REST API Walkthrough
ProxyWhirl exposes a FastAPI server for remote control. The following cell spins up the ASGI app in-process using FastAPI's `TestClient`, adds a proxy, retrieves system status, and issues a proxied request. We patch `httpx.AsyncClient.request` so the example remains network-free.


In [ ]:
try:
    from fastapi.testclient import TestClient
except ImportError as exc:  # pragma: no cover - optional dependency guidance
    raise ImportError(
        "FastAPI is not installed. Install the API extras with `pip install proxywhirl[storage]` "
        "or `pip install proxywhirl[api]` to run this section."
    ) from exc

import httpx
from unittest.mock import patch

from proxywhirl.api import app

async def mock_async_request(self, method, url, *args, **kwargs):
    return httpx.Response(200, json={"requested": url, "method": method, "origin": "198.51.100.42"})

with TestClient(app) as client:
    added = client.post(
        "/api/v1/proxies",
        json={"url": "http://proxy-api.local:8000"},
    )
    print("Add proxy →", added.json())

    status_response = client.get("/api/v1/status").json()
    print("Status summary keys:", list(status_response["data"].keys()))

    with patch.object(httpx.AsyncClient, "request", new=mock_async_request):
        proxied = client.post(
            "/api/v1/request",
            json={
                "url": "https://example.org/json",
                "method": "GET",
                "timeout": 10,
            },
        )
        print("Proxied request →", proxied.json()["data"])  # type: ignore[index]


## 7. Advanced Patterns & Utilities
A few more building blocks worth knowing about when you step into production scenarios.

### Background health monitoring
```python
import asyncio
from proxywhirl import HealthMonitor, HealthStatus, Proxy, ProxyPool

async def monitor_demo():
    pool = ProxyPool(
        name="monitored",
        proxies=[Proxy(url="http://proxy-monitor.local:8000", health_status=HealthStatus.HEALTHY)],
    )
    monitor = HealthMonitor(pool=pool, check_interval=30, failure_threshold=3)
    await monitor.start()
    try:
        await asyncio.sleep(120)  # Replace with real application runtime
    finally:
        await monitor.stop()

# asyncio.run(monitor_demo())  # Enable once you implement custom health checks.
```

### Browser rendering for JavaScript-heavy sources
Requires Playwright (`uv pip install 'proxywhirl[js]'`) and a downloaded browser binary (`playwright install chromium`).
```python
import asyncio
from proxywhirl.browser import BrowserRenderer

async def render_example():
    async with BrowserRenderer() as renderer:
        html = await renderer.render("https://js-heavy.example/proxies")
        print(html[:500])

# asyncio.run(render_example())
```

### Built-in source catalogs
```python
from proxywhirl import RECOMMENDED_SOURCES, ALL_HTTP_SOURCES
print("Recommended sources:", [source.url for source in RECOMMENDED_SOURCES[:3]])
print("Total HTTP sources:", len(ALL_HTTP_SOURCES))
```


## 8. Where to Go Next
- Populate a real proxy list and swap the mock transports for genuine traffic.
- Extend the FastAPI server with authentication and persistent storage by setting `PROXYWHIRL_REQUIRE_AUTH=true` and `PROXYWHIRL_STORAGE_PATH=...`.
- Hook the CLI into your CI/CD pipelines by emitting JSON (`--format json`) and parsing the structured output.
- Explore the comprehensive test suite in `tests/` for additional patterns, especially around geo-targeting and session persistence.

Happy proxying! 🌀
